In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score,accuracy_score

In [ ]:
df = pd.read_csv('/kaggle/input/latest-data-science-job-salaries-2024/DataScience_salaries_2025.csv')
df

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.isnull().sum()

In [ ]:
df.duplicated().sum()

In [ ]:
df_final = df.drop_duplicates() # drop duplicates values
df_final = df_final.dropna() # drop null or NaN values
df_final

In [ ]:
df['work_year'].value_counts()

In [ ]:
df['experience_level'].value_counts()

In [ ]:
df['salary_currency'].value_counts()

In [ ]:
def unique(df):
  for i in df.columns:
    print(df[i].unique())

In [ ]:
unique(df_final)

In [ ]:
df_final.select_dtypes(include=['object']).columns

In [ ]:
object_df = df_final.select_dtypes(include=['object'])
object_df

In [ ]:
def lableEncoderColumns(df):
  for i in df.columns:
    if df[i].dtype == 'object':
        df[i] = LabelEncoder().fit_transform(df[i])
  return df

In [ ]:
def OneHotEncoderColumns(df):
  for i in df.columns:
    if df[i].dtype == 'object':
        df[i] = OneHotEncoder().fit_transform(df[i])
  return df

In [ ]:
# Convert the 'product' column to category dtype
def encodeStringColumns(df):
  for i in df.columns:
    df[i] = df[i].astype('category')
    df[i] = df[i].cat.codes
  return df

In [ ]:
lableEncoderColumns(df_final)
df_final

In [ ]:
plt.figure(figsize=(15, 10))
sns.boxplot(data=df_final)
plt.xticks(rotation=90)
plt.title('Box Plots for Outlier Detection')
plt.show()

In [ ]:
def outlier(df, col):
  rows = df[col]
  Q1 = np.percentile(rows, 25)
  Q3 = np.percentile(rows, 75)
  IQR = Q3 - Q1
  lower_bound = Q1 - 1.5 * IQR
  upper_bound = Q3 + 1.5 * IQR
  filtered_rows = (rows >= lower_bound) & (rows <= upper_bound)
  return df.loc[filtered_rows]

In [ ]:
salary = df_final['salary']
Q1 = np.percentile(salary, 25)
Q3 = np.percentile(salary, 75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
filtered_salary = (salary >= lower_bound) & (salary <= upper_bound)
#filtered_salary = (salary >= lower_bound) & (salary <= upper_bound)
df_final = df_final.loc[filtered_salary]
#print(filtered_salary)

In [ ]:
salary = df_final['salary_in_usd']
Q1 = np.percentile(salary, 25)
Q3 = np.percentile(salary, 75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
filtered_salary = (salary >= lower_bound) & (salary <= upper_bound)
#filtered_salary = (salary >= lower_bound) & (salary <= upper_bound)
df_final = df_final.loc[filtered_salary]
#print(filtered_salary)

In [ ]:
df_final

In [ ]:
df_final = outlier(df_final,'salary')
df_final = outlier(df_final,'salary_in_usd')

In [ ]:
plt.figure(figsize=(15, 10))
sns.boxplot(data=df_final)
plt.xticks(rotation=90)
plt.title('Box Plots for Outlier Detection')
plt.show()

In [ ]:
plt.figure(figsize=(15,10))
sns.heatmap(df_final.corr(), annot=True, cmap='coolwarm')
plt.show()

In [ ]:
df_final.drop('salary', axis=1, inplace=True) # leads to overfittig becouse it contain multi currency
df_final.drop(['salary_currency'], axis = 1, inplace = True)
df_final.drop(['work_year'], axis = 1, inplace = True)
df_final.drop(['employment_type'], axis = 1, inplace = True)
df_final.drop(['remote_ratio'], axis = 1, inplace = True)
df_final.drop(['company_size'], axis = 1, inplace = True)
df_final.drop(['company_location'], axis = 1, inplace = True)

In [ ]:
df_final

In [ ]:
plt.figure(figsize=(15,5))
sns.heatmap(df_final.corr(), annot=True, cmap='coolwarm')
plt.show()

In [ ]:
X = df_final.drop('salary_in_usd', axis=1)
y = df_final['salary_in_usd']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)
y_pred

In [ ]:
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)
#acc = accuracy_score(y_test, y_pred)

accuracy_percentage = 100 - (abs(y_test - y_pred) / y_test * 100)
acc = accuracy_percentage.mean()

print(f'MAE: {mae}')
print(f'MSE: {mse}')
print(f'RMSE: {rmse}')
print(f'R-squared: {r2}')
print(f"Accuracy: {acc:.2f}")

In [ ]:
degree = 7
poly_reg = PolynomialFeatures(degree=degree)
x_poly = poly_reg.fit_transform(X_train)
x_test_poly = poly_reg.transform(X_test)

In [ ]:
poly_reg_model = LinearRegression()
poly_reg_model.fit(x_poly, y_train)

In [ ]:
poly_pred = poly_reg_model.predict(x_test_poly)
poly_pred

In [ ]:
mae = mean_absolute_error(y_test, poly_pred)
mse = mean_squared_error(y_test, poly_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, poly_pred)
#acc = accuracy_score(y_test, poly_pred)

accuracy_percentage = 100 - (abs(y_test - poly_pred) / y_test * 100)
acc = accuracy_percentage.mean()

print(f'MAE: {mae}')
print(f'MSE: {mse}')
print(f'RMSE: {rmse}')
print(f'R-squared: {r2}')
print(f"Accuracy: {acc:.2f}")